In [1]:
import pandas as pd
import numpy as np

fullData = pd.read_csv('PPMI_Original_Cohort_BL_to_Year_5_Dataset_Apr2020 (version 1).csv', sep=';') 
fullData

#filtering pacients from health control
fullData = fullData.loc[fullData['APPRDX'] != 2]

In [2]:
df = fullData[['PATNO','cogstate', 'age_cat', 'NHY', 'rigidity', 'upsit', 'tremor', 'gen', 'race', 'updrs_totscore', 'ess_cat',
               'gds_cat', 'scopa', 'bjlot', 'moca', 'NP1APAT', 'NP1DDS', 'NP1HALL', 'DOMSIDE', 'sft', 'stai', 'APPRDX']]

In [3]:
dfFirst = df.drop_duplicates(['PATNO'],keep='first')
#dfFirst

In [4]:
dfLast = df.drop_duplicates(['PATNO'],keep='last')
#dfLast

In [5]:
##Getting max value of NHY and adding to a new Column and calc new attributte 'NHY_EVOL' for a given pacient 
# and adding both to dfGrouped
dfGrouped = dfFirst.copy()
dfGrouped.index = fullData[['PATNO','NHY']].groupby('PATNO')['NHY'].agg(pd.Series.max).to_frame().index
dfGrouped['NHY_MAX'] = fullData[['PATNO','NHY']].groupby('PATNO')['NHY'].agg(pd.Series.max).to_frame()
dfGrouped['NHY_EVOL'] = dfGrouped['NHY_MAX'] - dfGrouped['NHY']
#dfGrouped

In [61]:
"""
This code is commented because this kind of generated field biases the data, so we cant use anymore
"""
##Getting max value of 'updrs_totscore' and adding to a new Column and calc new attributte 'updrs_totscore' for a given pacient 
# and adding both to dfGrouped

#dfGrouped.index = fullData[['PATNO','updrs_totscore']].groupby('PATNO')['updrs_totscore'].agg(pd.Series.max).to_frame().index
#dfGrouped['updrs_max'] = fullData[['PATNO','updrs_totscore']].groupby('PATNO')['updrs_totscore'].agg(pd.Series.max).to_frame()
#dfGrouped['updrs_evol'] = dfGrouped['updrs_max'] - dfGrouped['updrs_totscore']
#dfGrouped  


'\nThis code is commented because this kind of generated field biases the data, so we cant use anymore\n'

In [6]:
"""
This code shows which variables need to be complete (has missing data)
"""

#dfGrouped.isnull().sum()

'\nThis code shows which variables need to be complete (has missing data)\n'

In [8]:
#Creating dummy variable to indicate if the data was completed
dfGrouped['cogstate_missed'] = dfGrouped['cogstate'].isnull().astype('int')
##Filling missing value with mode
dfGrouped['cogstate'] = dfGrouped['cogstate'].fillna(df['cogstate'].mode()[0])
dfGrouped['cogstate'].isnull().sum()

dfGrouped['updrs_totscore'] = dfGrouped['updrs_totscore'].fillna(dfGrouped['updrs_totscore'].median())
dfGrouped['updrs_totscore'].isnull().sum()

dfGrouped['rigidity'] = dfGrouped['rigidity'].fillna(dfGrouped['rigidity'].median())
dfGrouped['rigidity'].isnull().sum()

dfGrouped['tremor'] = dfGrouped['tremor'].fillna(dfGrouped['tremor'].median())
dfGrouped['tremor'].isnull().sum()

dfGrouped['NHY'] = dfGrouped['NHY'].fillna(df['NHY'].mode()[0])
dfGrouped['NHY'].isnull().sum()

#BJLOT Benton Judgement of Line Orientation Score - Teste de habilidade Visoespacial associado com doenças neurológicas
dfGrouped['bjlot'] = dfGrouped['bjlot'].fillna(dfGrouped['bjlot'].median())
dfGrouped['bjlot'].isnull().sum()

#MONTREAL COGNITIVE ASSESSMENT (MOCA)
dfGrouped['moca'] = dfGrouped['moca'].fillna(dfGrouped['moca'].median())
dfGrouped['moca'].isnull().sum()

#Categorical Epworth Sleepiness Scale Score: 1 sonolento  - 0 normal
dfGrouped['ess_cat'] = dfGrouped['ess_cat'].fillna(df['ess_cat'].mode()[0])
dfGrouped['ess_cat'].isnull().sum()

#NP1APAT 0-Normal 1-Slight 2-Mild 3-Moderate 4-Severe
dfGrouped['NP1APAT'] = dfGrouped['NP1APAT'].fillna(df['NP1APAT'].mode()[0])
dfGrouped['NP1APAT'].isnull().sum()

#NP1DDS  0-Normal 1-Slight 2-Mild 3-Moderate 4-Severe
dfGrouped['NP1DDS'] = dfGrouped['NP1DDS'].fillna(df['NP1DDS'].mode()[0])
dfGrouped['NP1DDS'].isnull().sum()

#NP1HALL 0-Normal 1-Slight 2-Mild 3-Moderate 4-Severe
dfGrouped['NP1HALL'] = dfGrouped['NP1HALL'].fillna(df['NP1HALL'].mode()[0])
dfGrouped['NP1HALL'].isnull().sum()

#DOMSIDE (1 Left,2 Right,3 Simetrica)     OBS: 196 instances filled. Think if we keep it or not 
dfGrouped['DOMSIDE_MISSED'] = dfGrouped['DOMSIDE'].isnull().astype('int')
dfGrouped['DOMSIDE'] = dfGrouped['DOMSIDE'].fillna(df['DOMSIDE'].mode()[0])
dfGrouped['DOMSIDE'].isnull().sum()

#SCOPA-AUT Total Score (The SCOPA-AUT was developed to evaluate autonomic symptoms)
dfGrouped['scopa_missed'] = dfGrouped['scopa'].isnull().astype('int')
dfGrouped['scopa'] = dfGrouped['scopa'].fillna(dfGrouped['scopa'].median())
dfGrouped['scopa'].isnull().sum()

#sft (Semantic Fluency Total Score)
dfGrouped['sft'] = dfGrouped['sft'].fillna(dfGrouped['sft'].median())
dfGrouped['sft'].isnull().sum()

#stai (State‐Trait Anxiety Inventory)
dfGrouped['stai'] = dfGrouped['stai'].fillna(dfGrouped['stai'].median())
dfGrouped['stai'].isnull().sum()

## Fixing dependable values after filling
dfGrouped['NHY_EVOL'] = dfGrouped['NHY_MAX'] - dfGrouped['NHY']
dfGrouped['NHY_EVOL'].isnull().sum()
#dfGrouped

0

In [9]:
#Setting the field 'disease_status'. It will be used as label
dfGrouped['disease_status'] = np.where(dfGrouped['NHY_EVOL']>0, 'Evolved', 'Not Evolved')

#Droping undesired fields
dfGrouped = dfGrouped.drop(['NHY_MAX', 'NHY_EVOL'], axis=1)
#dfGrouped

In [11]:
dfGrouped['disease_status'].value_counts()

Not Evolved    291
Evolved        196
Name: disease_status, dtype: int64

In [37]:
#Saves the new generated csv after transformation process
dfGrouped.to_csv('processed_parkinson_data_without_healthy_contro_csv', index=False)